In [ ]:
import os

In [ ]:
from pathlib import Path

In [ ]:
from astropy.table import Table
import astropy.units as u
import matplotlib.pyplot as plt
import arya
import numpy as np

In [ ]:
import tomllib

In [ ]:
from astropy.coordinates import SkyCoord
import sys
sys.path.append("..")
sys.path.append("../../imaging")

from phot_utils import xmatch, get_atm_extinction, to_mag


In [ ]:
airmass = 1.2

In [ ]:
def get_zeropoint(filt, exposure=190, gain=1.9):
    d = None
    with open(f"../std1/img_{filt}_11/flat_fielded-astrom-zeropoint.toml", "rb") as f:
        d = tomllib.load(f)

    return d["zeropoint"] + d["ap_corr"] + 2.5 * np.log10(exposure) - get_atm_extinction(airmass, f"Sloan_{filt}")[0] - 2.5 * np.log10(gain)

In [ ]:
def get_airmass(file):
    path = Path(file).parent
    img = CCDData.read(path / "flat_fielded.fits")
    return img.header["AIRMASS"]

In [ ]:
def calibrate_mag(cat, filt):
    idx_aper = 3

    filt_bad = (cat["MAG_APER"][:, idx_aper] > 50) | (cat["FLAGS"] & 64 + 128 + 32 + 16 > 0) 
    cat["MAG"] = cat["MAG_APER"][:, idx_aper] + get_zeropoint("g") 
    cat["MAG_ERR"] = cat["MAGERR_APER"][:, idx_aper]

    cat["MAG"][filt_bad] = np.nan
    cat["MAG_ERR"][filt_bad] = np.nan
        

In [ ]:
objname = "yasone2"

In [ ]:
filename_g = "../yasone2/stacked_g/detection.cat"
filename_r = "../yasone2/stacked_r/detection.cat"
filename_i = "../yasone2/stacked_i/detection.cat"


In [ ]:
cat_g = Table.read(filename_g, hdu=2)
cat_r = Table.read(filename_r, hdu=2)
cat_i = Table.read(filename_i, hdu=2)

In [ ]:
calibrate_mag(cat_g, "g")
calibrate_mag(cat_r, "r")
calibrate_mag(cat_i, "i")

In [ ]:
plt.scatter(cat_g["ALPHA_J2000"], cat_g["DELTA_J2000"], s=4)
plt.scatter(cat_r["ALPHA_J2000"], cat_r["DELTA_J2000"], s=1)
plt.scatter(cat_i["ALPHA_J2000"], cat_i["DELTA_J2000"], s=0.25)


In [ ]:
xmatch_idx_gr, xmatch_sep_gr, xmatch_filt_gr = xmatch(cat_r["ALPHA_J2000"], cat_r["DELTA_J2000"], cat_g["ALPHA_J2000"], cat_g["DELTA_J2000"], 1*u.arcsec)
xmatch_idx_ir, xmatch_sep_ir, xmatch_filt_ir = xmatch(cat_r["ALPHA_J2000"], cat_r["DELTA_J2000"], cat_i["ALPHA_J2000"], cat_i["DELTA_J2000"],  1*u.arcsec)

In [ ]:
xmatch_filt_gr, xmatch_filt_ir

In [ ]:
np.sum(xmatch_filt_gr & xmatch_filt_ir)

In [ ]:
len(cat_r), len(cat_g), len(cat_i)

In [ ]:
filt = xmatch_filt_gr & xmatch_filt_ir
catalogue_matched = cat_r.copy()

for col in ["MAG", "MAG_ERR", "FLAGS", "FLAGS_WEIGHT"]:
    catalogue_matched[f"G{col}"] = np.where(xmatch_filt_gr, cat_g[col][xmatch_idx_gr], np.nan)
    catalogue_matched[f"I{col}"] = np.where(xmatch_filt_ir, cat_i[col][xmatch_idx_ir], np.nan)
    catalogue_matched.rename_column(col, f"R{col}")
    


In [ ]:

x = catalogue_matched["MAG_APER"][:, i] - catalogue_matched["MAG_APER"][:, j]
y = -catalogue_matched["MAG_APER"][:, k] + catalogue_matched["MAG_APER"][:, l]
catalogue_matched["APER_COEFF"] = y - 0.5*x
catalogue_matched["LIKELY_STAR"] = catalogue_matched["APER_COEFF"] < 0.05

In [ ]:
catalogue_matched.write(f"../{objname}/allcolours.cat", format="fits", overwrite=True)

In [ ]:
plt.scatter(catalogue_matched["ALPHA_J2000"], catalogue_matched["DELTA_J2000"], s=0.3)
plt.scatter(cat_g["ALPHA_J2000"][xmatch_idx_gr[filt]], cat_g["DELTA_J2000"][xmatch_idx_gr[filt]], s=0.2)
plt.scatter(cat_i["ALPHA_J2000"][xmatch_idx_ir[filt]], cat_i["DELTA_J2000"][xmatch_idx_ir[filt]], s=0.1)

In [ ]:
plt.scatter(catalogue_matched["GMAG"] - catalogue_matched["RMAG"], catalogue_matched["GMAG"], s=1)
plt.ylim(25, 16)
plt.xlim(-0.5, 2.5)

In [ ]:
plt.scatter(catalogue_matched["RMAG"], catalogue_matched["RMAG_ERR"])
plt.ylim(0, 1)


In [ ]:
plt.scatter(catalogue_matched["GMAG"] - catalogue_matched["IMAG"], catalogue_matched["IMAG"], s=1)
plt.ylim(26, 16)
plt.xlim(-0.5, 2.5)

In [ ]:
plt.scatter(catalogue_matched["RMAG"] - catalogue_matched["IMAG"], catalogue_matched["RMAG"], s=1)
plt.ylim(25, 16)
plt.xlim(-1.5, 1.5)

In [ ]:
plt.scatter(catalogue_matched["GMAG"] - catalogue_matched["IMAG"], catalogue_matched["IMAG"], s=1)
plt.ylim(25, 16)
plt.xlim(-1.5, 1.5)

In [ ]:
c = plt.scatter(catalogue_matched["GMAG"] - catalogue_matched["RMAG"], catalogue_matched["RMAG"] - catalogue_matched["IMAG"], s=1, c=np.log10(catalogue_matched["FLUX_RADIUS"]))
plt.xlim(-0.5, 2)
plt.ylim(-1, 3)
plt.colorbar()

In [ ]:
plt.xlim(-2, 1)
plt.axvline(0.05)



In [ ]:
plt.xlim(-2, 1)
plt.axvline(0.05)



In [ ]:
plt.scatter(flux_param, (tab["FLUX_RADIUS"]), s=0.3, lw=0)
plt.scatter(flux_param, (tab["KRON_RADIUS"]), s=0.3, lw=0)
plt.scatter(flux_param, 2.5*(tab["FWHM_WORLD"].to("arcsec")), s=0.3, lw=0)
plt.scatter(flux_param, 3*(tab["ELLIPTICITY"]), s=0.1, lw=0)

plt.axvline(0.05)
plt.xlim(-2, 1)
plt.ylim(0, 10)

In [ ]:
plt.scatter(y, (tab["FLUX_RADIUS"]), s=0.3, lw=0)
plt.scatter(y, (tab["KRON_RADIUS"]), s=0.3, lw=0)
plt.scatter(y, 2.5*(tab["FWHM_WORLD"].to("arcsec")), s=0.3, lw=0)
plt.scatter(y, (tab["ELLIPTICITY"]), s=0.1, lw=0)

plt.axvline(0.05)
plt.xlim(-0.5, 2)
plt.ylim(0, 10)

In [ ]:
i = 0
j = 1
k = 3
l = 1

tab = catalogue_matched[catalogue_matched["RFLAGS"] & 4 == 0]

x = tab["MAG_APER"][:, i] - tab["MAG_APER"][:, j]
y = -tab["MAG_APER"][:, k] + tab["MAG_APER"][:, l]
flux_param = y - 0.5*x

c = plt.scatter(tab["RMAG"],  flux_param, s=0.3, alpha=1, c=np.log10(tab["FLUX_RADIUS"]), lw=0, label="all sources", vmax=0.5)

plt.colorbar(c)
plt.axhline(0.05)
plt.axhline(-0.3)

plt.ylim(-1.1, 0.5)
plt.ylim(-2, 2)
# plt.xlim(1.8, 0.7)
plt.xlabel(f"I{i} - I{j}")
plt.ylabel(f"I{l} - I{k}")

In [ ]:
c = plt.scatter(x,  flux_param, s=0.3, alpha=1, c=np.log10(tab["FLUX_RADIUS"]), lw=0, label="all sources", vmax=1)

plt.colorbar(c)
plt.axhline(0.1)
plt.axhline(-0.3)

plt.ylim(-1.1, 0.5)
plt.ylim(-2, 2)
plt.xlim(1.8, 0.7)
plt.xlabel(f"I{i} - I{j}")
plt.ylabel(f"I{l} - I{k}")

##### i = 1
j = 0
k = 3
l = 1

tab = catalogue_matched
plt.scatter(tab["FLUX_RADIUS"] , -tab["MAG_APER"][:, k] + tab["MAG_APER"][:, l], s=0.3, alpha=1, c="black", lw=0)
plt.ylim(-5)
plt.axhline(0.35)
plt.xlim(0, 10)